In [1]:
import os
import cv2
import tkinter as tk
from tkinter import simpledialog
import mediapipe as mp

import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.models import save_model



from tensorflow.keras.models import load_model


# 1.DATA COLLECTION UTPTO 1000 images per label
#### here we detct the input from users camera and save the images in the respective folder
#### also the part of image containing hand is saved by cropping it 
#### the only problem in this is with gestures requiring two hands
#### to solve this issue a diffrent approach need to be seen.

In [9]:
def cptr_crop(nm_img):
    mp_hands = mp.solutions.hands
    hands = mp_hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.5)
    mp_drawing = mp.solutions.drawing_utils
    
    
    fldName = "CollectedData"
    if not os.path.exists(fldName):
        os.makedirs(fldName)
    
    cap = cv2.VideoCapture(0)
    print("Press 's' to save the cropped hand image with label, or 'q' to quit.")
    
    img_cnt = 0
    
    while img_cnt < nm_img:
        ret, frame = cap.read()
        if not ret:
            print("Failed to capture image")
            break
        
    
        img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        rslt = hands.process(img_rgb)
        
       
        if rslt.multi_hand_landmarks:
            for hand_landmarks in rslt.multi_hand_landmarks:
                mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
        
        
        cv2.imshow("Capture Image with Landmarks", frame)
        #caputerr the imae with label and landmarkkk
        
        
        
        
        key = cv2.waitKey(1) & 0xFF
        if key == ord('s') and img_cnt < nm_img:  
          
            root = tk.Tk()
            root.withdraw()
            label = simpledialog.askstring("Input", "Enter the label for this image:")
            root.destroy()
            
            
            # Take lanbel inpute from the user ....
            #The only problem is that for a single label user had to enter same text multiple time 
            # problem need to be considered i think it can be resolved using the previously used GUI interface ... handeled later
            
            
            if label and rslt.multi_hand_landmarks:
                
                label_dir = os.path.join(fldName, label)
                
                if not os.path.exists(label_dir):
                    os.makedirs(label_dir)
                
                for idx, hand_landmarks in enumerate(rslt.multi_hand_landmarks):
                    
                    img_h, img_w, _ = frame.shape
                    x_min, y_min = img_w, img_h
                    x_max, y_max = 0, 0
                    
                    for landmark in hand_landmarks.landmark:
                        x, y = int(landmark.x * img_w), int(landmark.y * img_h)
                        x_min, y_min = min(x, x_min), min(y, y_min)
                        x_max, y_max = max(x, x_max), max(y, y_max)
                    
                    
                    padding = 20
                    x_min = max(0, x_min - padding)
                    y_min = max(0, y_min - padding)
                    x_max = min(img_w, x_max + padding)
                    y_max = min(img_h, y_max + padding)
                    
                    
                    cropped_hand = frame[y_min:y_max, x_min:x_max]
                    
                    
                    file_count = len(os.listdir(label_dir)) + 1 
                    file_name = os.path.join(label_dir, f"{file_count}.jpg")
                    cv2.imwrite(file_name, cropped_hand)
                    print(f"Cropped hand image saved as {file_name}")
                    img_cnt += 1
                    
                    
                    if img_cnt >= nm_img:
                        break
            else:
                print("No hand detected or label not provided.")
        
        elif key == ord('q'):  
            break

    cap.release()
    cv2.destroyAllWindows()


cptr_crop(nm_img=1000) 
# The only isssue with this part was saving multiple images with same label was not possible 
# so the thought was to make separate folder for each label and insert a multiple images in that folder 
# the labels will be later handeled using label encoder or any other technique 

# Code runnuhgg fine with no errors debugging not done

Press 's' to save the cropped hand image with label, or 'q' to quit.


c:\Users\KIIT\AppData\Local\Programs\Python\Python311\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


Cropped hand image saved as CollectedData\thank you\1.jpg
Cropped hand image saved as CollectedData\thank you\2.jpg
Cropped hand image saved as CollectedData\thanks\1.jpg
Cropped hand image saved as CollectedData\thanks\2.jpg
Cropped hand image saved as CollectedData\thnkss\1.jpg
Cropped hand image saved as CollectedData\thnkss\2.jpg


# 2. Loading the data and using label encoder

In [10]:
def load_data(fldName):
    images = []
    labels = []
    
    for label in os.listdir(fldName):
        label_dir = os.path.join(fldName, label)
        for img_file in os.listdir(label_dir):
            img_path = os.path.join(label_dir, img_file)
            img = cv2.imread(img_path)
            if img is None:
                print(f"Image at {img_path} is not valid and will be skipped.")
                continue
            
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  
            img = cv2.resize(img, (128, 128)) 
            images.append(img)
            labels.append(label)
    
    images = np.array(images) / 255.0 
    labels = np.array(labels)

   
    label_encoder = LabelEncoder()
    labels = label_encoder.fit_transform(labels)
    np.save('label_class.npy', label_encoder.classes_)
    
    
    x_train, x_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)
    
    return x_train, x_test, y_train, y_test

# LAbel encoder seems fine for now no error saved label of encoder so that we can revert back them to respective labels 

# 3. Building a MODEL

In [11]:
def build_model():
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
        MaxPooling2D(pool_size=(2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dense(len(np.unique(y_train)), activation='softmax')  # Number of output classes
    ])
    
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model


fldName = "CollectedData"
x_train, x_test, y_train, y_test = load_data(fldName)


model = build_model()


history = model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=20, batch_size=32)
model_save_path = 'models/hand_gesture_recognition_model.h5'


model.save(model_save_path)

print(f"Model saved to {model_save_path}")

Epoch 1/20


c:\Users\KIIT\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - accuracy: 0.0500 - loss: 2.2062 - val_accuracy: 0.1667 - val_loss: 3.8818
Epoch 2/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 261ms/step - accuracy: 0.2000 - loss: 2.6209 - val_accuracy: 0.0000e+00 - val_loss: 2.7711
Epoch 3/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 273ms/step - accuracy: 0.2500 - loss: 2.0631 - val_accuracy: 0.1667 - val_loss: 2.3378
Epoch 4/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 271ms/step - accuracy: 0.5500 - loss: 1.9984 - val_accuracy: 0.1667 - val_loss: 2.2005
Epoch 5/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 220ms/step - accuracy: 0.5500 - loss: 1.8915 - val_accuracy: 0.0000e+00 - val_loss: 2.1899
Epoch 6/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 200ms/step - accuracy: 0.7000 - loss: 1.6575 - val_accuracy: 0.1667 - val_loss: 2.3460
Epoch 7/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 287ms/step - accuracy: 0.5500 - loss: 1.3960 - val_accuracy: 0.1667 - val_loss: 2.2127
Epoch 8/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 288ms/step - accuracy: 0.8500 - loss: 1.0263 - val_accuracy: 0.5000 - val_loss: 1.922

Model saved to models/hand_gesture_recognition_model.h5


# FINAL PREDICTION REAL TIME

In [12]:
model = load_model('models/hand_gesture_recognition_model.h5')
label_encoder = LabelEncoder()
label_encoder.classes_ = np.load('label_class.npy')  

# Loading the saved labels and saved model
# or else we would have to train it every time we use it 
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils

def prdct_gstr():
   
    cap = cv2.VideoCapture(0)
    print("Press 'q' to quit.")
    
    while True:
        ret, frame = cap.read()
        if not ret:
            print("Failed to capture image")
            break
        
       
        img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        rslt = hands.process(img_rgb)
        
        if rslt.multi_hand_landmarks:
            for hand_landmarks in rslt.multi_hand_landmarks:
                mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
                
                
                img_h, img_w, _ = frame.shape
                x_min, y_min = img_w, img_h
                x_max, y_max = 0, 0
                for landmark in hand_landmarks.landmark:
                    x, y = int(landmark.x * img_w), int(landmark.y * img_h)
                    x_min, y_min = min(x, x_min), min(y, y_min)
                    x_max, y_max = max(x, x_max), max(y, y_max)

                padding = 20
                x_min = max(0, x_min - padding)
                y_min = max(0, y_min - padding)
                x_max = min(img_w, x_max + padding)
                y_max = min(img_h, y_max + padding)
                
                cropped_hand = frame[y_min:y_max, x_min:x_max]
                cropped_hand = cv2.cvtColor(cropped_hand, cv2.COLOR_BGR2RGB) 
                cropped_hand = cv2.resize(cropped_hand, (128, 128))  
                cropped_hand = np.expand_dims(cropped_hand, axis=0) / 255.0  
                
             
                predictions = model.predict(cropped_hand)
                gesture_label_encoded = np.argmax(predictions, axis=1)[0]
                gesture_label = label_encoder.inverse_transform([gesture_label_encoded])[0]
                
                
                cv2.putText(frame, f'Gesture: {gesture_label}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

     
        cv2.imshow("Real-time Hand Gesture Prediction", frame)
        
        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):  # Quit
            break

    cap.release()
    cv2.destroyAllWindows()

prdct_gstr()

Press 'q' to quit.


c:\Users\KIIT\AppData\Local\Programs\Python\Python311\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 207ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step


c:\Users\KIIT\AppData\Local\Programs\Python\Python311\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step


...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step


c:\Users\KIIT\AppData\Local\Programs\Python\Python311\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step


c:\Users\KIIT\AppData\Local\Programs\Python\Python311\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
